<a href="https://colab.research.google.com/github/DmitryKutsev/ml_hw/blob/master/final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!unzip '/content/drive/My Drive/jigsaw-toxic-comment-train.csv.zip'

Archive:  /content/drive/My Drive/jigsaw-toxic-comment-train.csv.zip
  inflating: jigsaw-toxic-comment-train.csv  


In [4]:
import pandas as pd
from string import punctuation
import numpy as np
import matplotlib.pyplot as plt 
from statsmodels.graphics.mosaicplot import mosaic
import statistics 
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
from collections import Counter
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [5]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
#print(stopwords.words('english'))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
my_train = pd.read_csv('jigsaw-toxic-comment-train.csv')

In [6]:
my_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223549 entries, 0 to 223548
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             223549 non-null  object
 1   comment_text   223549 non-null  object
 2   toxic          223549 non-null  int64 
 3   severe_toxic   223549 non-null  int64 
 4   obscene        223549 non-null  int64 
 5   threat         223549 non-null  int64 
 6   insult         223549 non-null  int64 
 7   identity_hate  223549 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 13.6+ MB


In [34]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)
my_train.head(10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  after removing the cwd from sys.path.


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,cluster,cluster_num
0,0000997932d777bf,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",0,0,0,0,0,0,non_toxic,40
1,000103f0d9cfb60f,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0,0,0,0,0,0,non_toxic,40
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",0,0,0,0,0,0,non_toxic,40
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.\n\nThere appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It's listed in the relevant form eg Wikipedia:Good_article_nominations#Transport """,0,0,0,0,0,0,non_toxic,40
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember what page that's on?",0,0,0,0,0,0,non_toxic,40
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the tools well. · talk """,0,0,0,0,0,0,non_toxic,40
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0,toxic sever_toxic obscene insult,15
7,00031b1e95af7921,"Your vandalism to the Matt Shirvington article has been reverted. Please don't do it again, or you will be banned.",0,0,0,0,0,0,non_toxic,40
8,00037261f536c51d,"Sorry if the word 'nonsense' was offensive to you. Anyway, I'm not intending to write anything in the article(wow they would jump on me for vandalism), I'm merely requesting that it be more encyclopedic so one can use it for school as a reference. I have been to the selective breeding page but it's almost a stub. It points to 'animal breeding' which is a short messy article that gives you no info. There must be someone around with expertise in eugenics? 93.161.107.169",0,0,0,0,0,0,non_toxic,40
9,00040093b2687caa,alignment on this subject and which are contrary to those of DuLithgow,0,0,0,0,0,0,non_toxic,40


### **Анализ всех дополнительных колонок тональности (как проявляется тот или иной тип токсичности, как в данных это представлено, какие есть пограничные случаи)**

In [8]:
len(my_train.loc[my_train['toxic']==1])

21384

In [9]:
def max_tox_len(toxic_type):
  toxic_type_df = my_train.loc[my_train[toxic_type] == 1]
  len_list = [len(sent) for sent in toxic_type_df['comment_text']]
  return max(len_list)


print('toxic', max_tox_len('toxic'))
print('severe_toxic', max_tox_len('severe_toxic'))
print('obscene', max_tox_len('obscene'))
print('threat', max_tox_len('threat'))
print('insult', max_tox_len('insult'))
print('identity_hate', max_tox_len('identity_hate'))


toxic 5000
severe_toxic 5000
obscene 5000
threat 5000
insult 5000
identity_hate 5000


In [10]:
def max_none_tox_len(none_toxic_type):
  toxic_type_df = my_train.loc[my_train[none_toxic_type] == 0]
  len_list = [len(sent) for sent in toxic_type_df['comment_text']]
  return max(len_list)


print('toxic', max_none_tox_len('toxic'))
print('severe_toxic', max_none_tox_len('severe_toxic'))
print('obscene', max_none_tox_len('obscene'))
print('threat', max_none_tox_len('threat'))
print('insult', max_none_tox_len('insult'))
print('identity_hate', max_none_tox_len('identity_hate'))

toxic 5000
severe_toxic 5000
obscene 5000
threat 5000
insult 5000
identity_hate 5000


In [11]:
#Получается совсем не информативно, попробую измерить среднее

In [12]:
def mean_tox_len(toxic_type):
  toxic_type_df = my_train.loc[my_train[toxic_type] == 1]
  len_list = [len(sent) for sent in toxic_type_df['comment_text']]
  return statistics.mean(len_list)


print('toxic', mean_tox_len('toxic'))
print('severe_toxic', mean_tox_len('severe_toxic'))
print('obscene', mean_tox_len('obscene'))
print('threat', mean_tox_len('threat'))
print('insult', mean_tox_len('insult'))
print('identity_hate', mean_tox_len('identity_hate'))

toxic 280.60409652076316
severe_toxic 505.4566768603466
obscene 280.9517298187809
threat 316.9100145137881
insult 272.2064755838641
identity_hate 314.43882853094


In [13]:
#тут немного более информативно, severe_toxic явно выделяется, например. Попробую взять не-токсичные:

In [14]:
def mean_none_tox_len(none_toxic_type):
  toxic_type_df = my_train.loc[my_train[none_toxic_type] == 0]
  len_list = [len(sent) for sent in toxic_type_df['comment_text']]
  return statistics.mean(len_list)


print('toxic', mean_none_tox_len('toxic'))
print('severe_toxic', mean_none_tox_len('severe_toxic'))
print('obscene', mean_none_tox_len('obscene'))
print('threat', mean_none_tox_len('threat'))
print('insult', mean_none_tox_len('insult'))
print('identity_hate', mean_none_tox_len('identity_hate'))

toxic 402.6911779981698
severe_toxic 389.999386245583
obscene 397.33287608379965
threat 391.2418065153011
insult 397.34023416334895
identity_hate 391.7447929838506


In [15]:
#Попробую выделить списки слов, характерных для конкретных типов.

In [16]:
def tokenize(sents):
    words = []
    for sent in sents:
      for word in sent.split():
        words.append(word.strip(string.punctuation))
    words = [word for word in words if word]
    
    return words

In [17]:
#  это просто на будущее более удобное разделение данных по типам
toxic_df = my_train.loc[my_train['toxic'] == 1]
severe_toxic_df = my_train.loc[my_train['severe_toxic'] == 1]
obscene_df =  my_train.loc[my_train['obscene'] == 1]
threat_df = my_train.loc[my_train['threat'] == 1]
insult_df = my_train.loc[my_train['insult'] == 1]
identity_hate_df = my_train.loc[my_train['identity_hate'] == 1]

In [18]:
stoplist = stopwords.words('english')
for w in ['would', 'page', 'wikipedia', 'want', 'think', 'time']:
  stoplist.append(w)
'would' in stoplist

True

In [19]:
def freq_words(toxic_type, n):
  toxic_type_df = my_train.loc[my_train[toxic_type] == 1]
  length = len(toxic_type_df)
  counter = Counter(tokenize(toxic_type_df['comment_text']))
  frequent_dict = {key:val for key, val in counter.items() if val > length*n and \
                   key.lower() not in stoplist}

  return frequent_dict

In [20]:
freq_words('toxic', 0.1)


{'DIE': 2760,
 'FUCK': 5952,
 'NIGGER': 3397,
 'SUCK': 2675,
 'ass': 2240,
 'fuck': 6151,
 'fucking': 2766,
 'gay': 2192,
 'know': 2523,
 'like': 4149,
 'shit': 2971}

In [21]:
freq_words('severe_toxic', 0.1)

{'ADMINS': 200,
 'ANAL': 342,
 'ASS': 762,
 'Ass': 271,
 'BASTARD': 205,
 'BITCH': 644,
 'BITCH!!FUCK': 310,
 'BLOCK': 266,
 'BOT': 217,
 'Bitch': 293,
 'BunkSteve': 278,
 'COCK': 442,
 'COCKSUCKER': 494,
 'COCKSUCKING': 267,
 'CUNT': 819,
 'CUNTS': 241,
 'Chester': 261,
 'CriminalWar': 279,
 'DAMN': 377,
 'DICK': 331,
 'DIE': 1565,
 'DIE!FUK': 402,
 'DOG': 363,
 'EAT': 204,
 'FACK': 232,
 'FAGGOT': 605,
 'FAGS': 207,
 'FAT': 243,
 'FUCK': 3088,
 'FUCKER': 523,
 'FUCKING': 1084,
 'Fuck': 1051,
 'Fucking': 212,
 'GAY': 250,
 'GO': 223,
 'Go': 924,
 'HOMELAND': 229,
 'HUGE': 421,
 'KILL': 630,
 'MEXICANS': 358,
 'MOTHERFUCKER': 253,
 'MOTHERFUCKERDIE': 287,
 'MOTHJER': 489,
 'MUST': 562,
 'MarcolFuck': 260,
 'NIGGER': 808,
 'NIGGERJEW': 424,
 'NIGGERS': 393,
 'Nigger': 367,
 'Notrhbysouthbanof': 228,
 'OFFFUCK': 360,
 "Pro-Assad.Hanibal911You're": 345,
 'RAPE': 362,
 'SECURITYFUCK': 227,
 'SHIT': 1508,
 'SHUT': 364,
 'SUCK': 2253,
 'TRAITOR': 405,
 'U': 939,
 'USELESS': 224,
 'VANDAL': 4

In [22]:
freq_words('obscene', 0.1)

{'CUNT': 1440,
 'DICKS': 1223,
 'DIE': 1468,
 'FUCK': 5940,
 'FUCKING': 1592,
 'Fuck': 1961,
 'Go': 1240,
 'NIGGER': 2990,
 'SHIT': 1688,
 'SUCK': 2535,
 'U': 1319,
 'ass': 2121,
 'bitch': 1511,
 'fuck': 6073,
 'fucking': 2651,
 'get': 1248,
 'know': 1376,
 'like': 2061,
 'shit': 2623,
 'stupid': 1264,
 'u': 1599}

In [23]:
freq_words('threat', 0.1)

{'BAN': 131,
 'BLANK': 94,
 'BLOCK': 150,
 'DI': 90,
 'DIE': 1055,
 'DIE!FUK': 310,
 'Die': 89,
 'EDIE': 90,
 'FUCKIN': 82,
 'FUCKING': 74,
 'FoReVeR': 76,
 'FooL': 73,
 "I'm": 73,
 'JIM': 157,
 'KILL': 526,
 'LiVe': 76,
 'MUST': 478,
 'NIGGGERS': 123,
 'PaTHeTiC': 76,
 'RVV': 96,
 'ReSPeCT': 76,
 'SuPeRTR0LL': 149,
 'TALK': 101,
 'TRAITOR': 313,
 'VANDAL': 314,
 'WALES': 156,
 'ass': 753,
 'balls': 92,
 'bitch': 81,
 'decapitate': 84,
 'die': 120,
 'fuck': 102,
 'fucking': 176,
 'get': 84,
 'going': 383,
 'kill': 275,
 'like': 69,
 'murder': 109,
 'rip': 87,
 'shit': 96,
 'tiny': 83,
 'u': 70,
 'you.I': 81}

In [24]:
freq_words('insult', 0.1)

{'CUNT': 1134,
 'DIE': 1465,
 'FAT': 1565,
 'FUCK': 4238,
 'FUCKING': 1516,
 'Fuck': 1179,
 'Go': 1235,
 'JEW': 1246,
 'NIGGER': 3033,
 'SUCK': 2380,
 'U': 1279,
 'ass': 1882,
 'bitch': 1458,
 'faggot': 1718,
 'fuck': 4670,
 'fucking': 2252,
 'gay': 1154,
 'hi': 1345,
 'know': 1361,
 'like': 2052,
 'moron': 1474,
 'nigger': 1593,
 'shit': 2025,
 'stupid': 1447,
 'u': 1525}

In [25]:
freq_words('identity_hate', 0.1)

{'Ass': 271,
 'Bitch': 286,
 'BunkSteve': 278,
 'CUNT': 595,
 'DIE': 772,
 'FAGGOT': 555,
 'FAT': 1314,
 'FRANCE': 217,
 'FUCK': 696,
 'FUCKING': 449,
 'GAY': 681,
 'HATE': 228,
 'HUGE': 422,
 'JEW': 1245,
 'KILL': 313,
 'MEXICANS': 358,
 'NIGGER': 3098,
 'NIGGERJEW': 424,
 'NIGGERS': 1113,
 'Nigger': 507,
 'Please!Nigga': 383,
 'SUCK': 551,
 'TOMMY2010': 227,
 'ass': 327,
 'bitch': 324,
 'fag': 246,
 'faggot': 724,
 'fuck': 751,
 'fucking': 499,
 'gay': 1334,
 'like': 451,
 'nigga': 227,
 'niggas': 359,
 'nigger': 1594,
 'nigger!Dumbass': 215,
 'shit': 358,
 'stupid': 425,
 'sucks!!!!!!!!!!!U.S.A': 223,
 'u': 302}

In [26]:
#посмотрим, сколько частотных слов вообще в каком типе получилось

In [27]:
print('severe_toxic', len(freq_words('severe_toxic', 0.1)))
print('toxic', len(freq_words('toxic', 0.1)))
print('obscene', len(freq_words('obscene', 0.1)))
print('threat', len(freq_words('threat', 0.1)))
print('insult', len(freq_words('insult', 0.1)))
print('identity_hate', len(freq_words('identity_hate', 0.1)))

severe_toxic 101
toxic 11
obscene 21
threat 43
insult 25
identity_hate 39


In [28]:
#и как они пересекаются между собой. Я полагаю, что uppercase так же несет в себе информацию о тональности, 
# и полагаю upper-lowercase разными словами.

In [29]:
toxic_set = set(freq_words('toxic', 0.1).keys())
severe_toxic_set = set(freq_words('severe_toxic', 0.1).keys())
obscene_set = set(freq_words('obscene', 0.1).keys())
threat_set = set(freq_words('threat', 0.1).keys())
insult_set = set(freq_words('insult', 0.1).keys())
identity_hate_set = set(freq_words('identity_hate', 0.1).keys())                                                 
# как toxic пересекается с остальными

In [30]:
toxic_set.intersection(severe_toxic_set, obscene_set, threat_set, insult_set, identity_hate_set)

{'DIE', 'ass', 'fuck', 'fucking', 'like', 'shit'}

In [31]:
#И посмотрим, как они выделяются

In [32]:
print('severe_toxic_set', severe_toxic_set.difference(toxic_set, obscene_set, threat_set, insult_set, identity_hate_set))
print('toxic_set', toxic_set.difference(severe_toxic_set, obscene_set, threat_set, insult_set, identity_hate_set))
print('threat_set', threat_set.difference(toxic_set, severe_toxic_set, obscene_set, insult_set, identity_hate_set))
print('insult_set', insult_set.difference(toxic_set, obscene_set, threat_set, severe_toxic_set, identity_hate_set))
print('identity_hate_set', identity_hate_set.difference(toxic_set, obscene_set, threat_set, insult_set, identity_hate_set))
print('obscene_set', obscene_set.difference(toxic_set, identity_hate_set, threat_set, insult_set, identity_hate_set))


severe_toxic_set {'CUNTS', 'veggietales', 'bitches.fuck', 'BITCH', 'MOTHERFUCKER', 'DICK', 'fat', 'Notrhbysouthbanof', 'MarcolFuck', 'BITCH!!FUCK', 'EAT', 'yourselfgo', 'COCKSUCKING', 'mother', 'sucks', "Pro-Assad.Hanibal911You're", 'ancestryFuck-off-Jewish', 'Fucking', 'COCK', 'BOT', 'Chester', 'GO', 'FUCKER', 'FAGS', 'ass!Fuck', 'CriminalWar', 'asshole', 'bastard', 'HOMELAND', 'little', 'cock', 'BASTARD', 'SHUT', 'OFFFUCK', 'ASS', "YOU!!'FUCK", 'suck', 'dick', 'MOTHERFUCKERDIE', 'MOTHJER', 'COCKSUCKER', 'RAPE', 'penis/////Small', 'dickhead', 'ADMINS', 'ANAL', 'piece', 'USELESS', 'fucksex', 'FACK', 'DAMN', 'DOG', 'SECURITYFUCK', 'fucked', 'cocks', 'pussy', 'cunt'}
toxic_set set()
threat_set {'kill', 'tiny', 'FUCKIN', "I'm", 'rip', 'FooL', 'JIM', 'DI', 'BAN', 'BLANK', 'RVV', 'decapitate', 'TALK', 'ReSPeCT', 'Die', 'you.I', 'balls', 'NIGGGERS', 'LiVe', 'die', 'EDIE', 'PaTHeTiC', 'murder', 'WALES', 'SuPeRTR0LL', 'FoReVeR'}
insult_set {'moron'}
identity_hate_set set()
obscene_set {'DICKS'

In [33]:
#в severe_toxic просто больше слов, а вот threat можно заметить слова с верхним и нижним регистром в одном.
# Попробую их тоже рассматривать как признак  и посчитать.

In [34]:
re.findall(r'([A-Z]+[a-z]+[A-Z]+)|([a-z]+[A-Z]+[a-z]+)', 'HellO')

[('HellO', '')]

In [35]:
def upper_lower_words(toxic_type):
  len_list = []
  token_list = []
  toxic_type_df = my_train.loc[my_train[toxic_type] == 1]
  length = len(toxic_type_df)
  df_tokens = tokenize(toxic_type_df['comment_text'])
  for token in df_tokens:
    if len(re.findall(r'([A-Z]+[a-z]+[A-Z]+)|([a-z]+[A-Z]+[a-z]+)', token)) > 0:
      len_list.append(len(token))
      token_list.append(token)
  return len_list

In [36]:
#еще потом придумаю, но минималка какая-то вроде есть.
# len_list = [len(sents) for sents in my_train['comment_text']]
# punkt_len_list = [len(re.findall(r'[\.,?!)\";}\]\*:@\'\({\[]+', sents)) for sents in my_train['comment_text']]
# upper_len_list = [len(re.findall(r'[A-Z]+', sents)) for sents in my_train['comment_text']]

In [37]:
all_toxic = " ".join(my_train.loc[my_train['toxic'] == 1]['comment_text'])
all_severe_toxic = " ".join(my_train.loc[my_train['severe_toxic'] == 1]['comment_text'])
all_obscene = " ".join(my_train.loc[my_train['obscene'] == 1]['comment_text'])
all_threat = " ".join(my_train.loc[my_train['threat'] == 1]['comment_text'])
all_insult = " ".join(my_train.loc[my_train['insult'] == 1]['comment_text'])
all_identity_hate = " ".join(my_train.loc[my_train['identity_hate'] == 1]['comment_text'])                                  

In [38]:
print('severe_toxic', sum(upper_lower_words('severe_toxic')))
print('toxic', sum(upper_lower_words('toxic')))
print('obscene', sum(upper_lower_words('obscene')))
print('threat', sum(upper_lower_words('threat')))
print('insult', sum(upper_lower_words('insult')))
print('identity_hate', sum(upper_lower_words('identity_hate')))

severe_toxic 19582
toxic 56533
obscene 36456
threat 5047
insult 31223
identity_hate 9179


In [39]:
print('severe_toxic', sum(upper_lower_words('severe_toxic'))/len(all_severe_toxic))
print('toxic', sum(upper_lower_words('toxic'))/len(all_toxic))
print('obscene', sum(upper_lower_words('obscene'))/len(all_obscene))
print('threat', sum(upper_lower_words('threat'))/len(all_threat))
print('insult', sum(upper_lower_words('insult'))/len(all_insult))
print('identity_hate', sum(upper_lower_words('identity_hate'))/len(all_identity_hate))

severe_toxic 0.01970680318456787
toxic 0.009388023988092638
obscene 0.01065063967819035
threat 0.02304155880916184
insult 0.010110011090154048
identity_hate 0.013745483188401023


In [40]:
# И так же стоит посмотреть на пунктуацию

In [41]:
def punkt_len(toxic_type):
  final_len = 0
  all_text = " ".join(my_train.loc[my_train[toxic_type] == 1]['comment_text'])
  all_punkt = re.findall(r'[\.,?!)\";}\]\*:@\'\({\[]+', all_text)
  for punkt_peace in all_punkt:
    final_len += len(punkt_peace)
  return final_len


In [42]:
print('toxic', punkt_len('toxic'))
print('severe_toxic', punkt_len('severe_toxic'))
print('obscene', punkt_len('obscene'))
print('threat', punkt_len('threat'))
print('insult', punkt_len('insult'))
print('identity_hate', punkt_len('identity_hate'))

toxic 269048
severe_toxic 51662
obscene 151698
threat 16543
insult 134153
identity_hate 25735


In [43]:
print('toxic', punkt_len('toxic')/len(all_toxic))
print('severe_toxic', punkt_len('severe_toxic')/len(all_severe_toxic))
print('obscene', punkt_len('obscene')/len(all_obscene))
print('threat', punkt_len('threat')/len(all_threat))
print('insult', punkt_len('insult')/len(all_insult))
print('identity_hate', punkt_len('identity_hate')/len(all_identity_hate))

toxic 0.04467884382481645
severe_toxic 0.051991260653720006
obscene 0.0443186509189741
threat 0.07552536306319879
insult 0.0434387572551464
identity_hate 0.03853796817229549


## **Бейзлайн модель из sklearn (векторайзер + модель) c подбором параметров в greed_search (как минимум 10 параметров) **

In [44]:
#https://github.com/mannefedov/hse_ml_m1/blob/master/1_intro_regression/pandas_regression.ipynb - регуляризация
# тут и статистики есть!!

#https://github.com/mannefedov/hse_ml_m1/blob/master/3_overfitting_validation/overfitting_validation.ipynb - permutation importance


#ансамбли - https://github.com/DmitryKutsev/ml_hw/blob/master/hw5.ipynb

#ох..

In [45]:
!pip install eli5

     |████████████████████████████████| 112kB 2.9MB/s 


In [46]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.svm import LinearSVR
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import roc_auc_score, roc_curve
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
import eli5
from eli5.sklearn import PermutationImportance
import warnings

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [47]:
cv = CountVectorizer(max_features=1000, min_df=5, max_df=0.4, ngram_range=(1,2))
X = cv.fit_transform(my_train['comment_text'])
y = my_train['toxic']
miniX = cv.fit_transform(my_train['comment_text'][:10000])
miniy = my_train['toxic'][:10000]

In [48]:

def scores(model):
  ''' Функция для оценки моделей на тесте'''
  
  predicted = model.predict(X_test)
  acc = accuracy_score(predicted, twenty_test.target)
  micro_f1 = f1_score(predicted, twenty_test.target, average = 'micro')
  micro_p = precision_score(predicted, twenty_test.target, average = 'micro')
  micro_r = recall_score(predicted, twenty_test.target, average = 'micro')
  macro_f1 = f1_score(predicted, twenty_test.target, average = 'macro')
  macro_p = precision_score(predicted, twenty_test.target, average = 'macro')
  macro_r = recall_score(predicted, twenty_test.target, average = 'macro')
  print('acc={0:1.4f}'.format(acc))
  print('micro F1={0:1.4f}, micro P={1:1.4f}, micro R={2:1.4f}'.format(micro_f1, micro_p, micro_r))
  print('macro F1={0:1.4f}, macro P={1:1.4f}, macro R={2:1.4f}\n'.format(macro_f1, macro_p, macro_r))

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
miniX_train, miniX_test, miniy_train, miniy_test = train_test_split(miniX, miniy, random_state=42)

In [50]:
# regressor = Lasso(alpha=1)
# regressor.fit(X_train, y_train)
# preds = regressor.predict(X_test)
# mean_absolute_error(y_test, preds)

In [51]:
lr = LogisticRegression()

In [31]:

parameter_grid = {'class_weight' : ['balanced', None],
                  'penalty' : ['l2', 'l1'],
                  'solver' : ['liblinear', 'saga'],
                  'C' : [0.001, 0.01, 0.08],
                  'max_iter': [2,10],
                  'warm_start' : [False, True],
                  'tol':[0.0001, 0.001],
                  'l1_ratio': [0, 1],
                  'verbose': [0, 1],
                  'intercept_scaling': [1, 2]
                 }

grid_search = GridSearchCV(lr, param_grid=parameter_grid, cv=10, scoring='roc_auc')
grid_search.fit(miniX_train, miniy_train)


In [77]:
print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))

Best score: 0.8934634778370368
Best parameters: {'C': 0.08, 'class_weight': None, 'intercept_scaling': 1, 'l1_ratio': 0, 'max_iter': 10, 'penalty': 'l2', 'solver': 'liblinear', 'tol': 0.001, 'verbose': 0, 'warm_start': False}


## **Использование кластеризационного алгоритма для разделение по токсичности (оценивание нужно сделать по специальным метрикам)**

In [9]:
from sklearn import metrics
from sklearn.datasets import make_blobs

from sklearn.cluster import AffinityPropagation, MeanShift, AgglomerativeClustering
from sklearn.decomposition import TruncatedSVD, NMF
from sklearn.cluster import AffinityPropagation, AgglomerativeClustering, DBSCAN, \
                            KMeans, MiniBatchKMeans, Birch, MeanShift, SpectralClustering
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score, \
                            silhouette_score

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD, NMF
from sklearn import metrics
from sklearn.datasets import make_blobs

In [10]:
cluster_list = []
for row in range(len(my_train)):
  cluster = ''
  if my_train['toxic'][row] == 1:
    cluster = 'toxic'
  if my_train['severe_toxic'][row]  == 1:
    cluster += ' sever_toxic'
  if my_train['obscene'][row]  == 1:
    cluster += ' obscene'
  if my_train['threat'][row]  == 1:
    cluster += ' threat'
  if my_train['insult'][row]  == 1:
    cluster += ' insult'
  if my_train['identity_hate'][row]  == 1:
    cluster += ' identity_hate'
  if cluster == '':
    cluster = 'non_toxic'
  cluster_list.append(cluster)


In [11]:
print(len(set(cluster_list)))
cluster_list[:5]

41


['non_toxic', 'non_toxic', 'non_toxic', 'non_toxic', 'non_toxic']

In [12]:
my_train['cluster'] = cluster_list
my_train.head(6)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,cluster
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,non_toxic
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,non_toxic
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,non_toxic
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,non_toxic
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,non_toxic
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0,non_toxic


In [13]:
list_of_clusters = list(set(my_train['cluster']))
val = 0
clusters_num_count = Counter()
for i in list_of_clusters:
  clusters_num_count[i] = val
  val += 1

In [14]:
cluster_num_list = []
for row in range(len(my_train)):
  num = clusters_num_count[my_train['cluster'][row]]
  cluster_num_list.append(num)

In [15]:
my_train['cluster_num'] = cluster_num_list
my_train.head(10)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,cluster,cluster_num
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,non_toxic,40
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,non_toxic,40
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,non_toxic,40
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,non_toxic,40
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,non_toxic,40
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0,non_toxic,40
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0,toxic sever_toxic obscene insult,15
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0,non_toxic,40
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0,non_toxic,40
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0,non_toxic,40


In [16]:
# Похоже, это все было не нужно, но оставлю на всякий случай. Кластеризация:

In [17]:
def cluster_metrics(cluster, sample, cluster_num, X, y):
  labels = cluster.labels_

  print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X[:10000], labels[:10000]))
  
 
  print("Homogeneity: %0.3f" % metrics.homogeneity_score(y, labels)) 
  print("Completeness: %0.3f" % metrics.completeness_score(y, labels))
  print("V-measure: %0.3f" % metrics.v_measure_score(y, labels)) 

  print("Adjusted Rand Index: %0.3f"
        % metrics.adjusted_rand_score(y, labels))
  print("Adjusted Mutual Information: %0.3f"
        % metrics.adjusted_mutual_info_score(y, labels))
  
  sample['cluster'] = cluster.labels_
  print(sample[sample.cluster==cluster_num].head(7))

In [18]:
miniX = my_train[:5000]


In [19]:
cv = CountVectorizer(max_features=2000, ngram_range=(1,3))
X_cv = cv.fit_transform(miniX['comment_text'])
svd = TruncatedSVD(50)
X_svd = svd.fit_transform(X_cv)
y = miniX['cluster']

# tfidf = TfidfVectorizer(max_features=500)
# X_tf = tfidf.fit_transform(sample1['title'])
# y_tf = sample1['category_name']

In [20]:
cluster = AffinityPropagation(damping=0.7, preference=-5, 
                              max_iter=400, verbose=2)


In [21]:
cluster2 = KMeans(n_clusters=41, max_iter=300, n_init=10)


In [22]:
cluster_svd1 = AffinityPropagation(damping=0.7, preference=-5, 
                              max_iter=400, verbose=2)



In [23]:
cluster_svd2 = KMeans(n_clusters=40, max_iter=300, n_init=10)


In [24]:
cluster.fit(X_cv)

Converged after 137 iterations.


AffinityPropagation(affinity='euclidean', convergence_iter=15, copy=True,
                    damping=0.7, max_iter=400, preference=-5, verbose=2)

In [25]:
cluster2.fit(X_cv)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=41, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [86]:
labels = cluster.labels_
print(set(labels))
labels2 = cluster2.labels_
print(set(labels2))

In [27]:
def metr(labels, X): 
  print("Silhouette Coefficient: %0.3f"% metrics.silhouette_score(X[:100], labels[:100]))
  print("Homogeneity: %0.3f" % metrics.homogeneity_score(y, labels))
  print("Completeness: %0.3f" % metrics.completeness_score(y, labels)) 
  print("V-measure: %0.3f" % metrics.v_measure_score(y, labels)) 

  print("Adjusted Rand Index: %0.3f"
        % metrics.adjusted_rand_score(y, labels))
  print("Adjusted Mutual Information: %0.3f"
        % metrics.adjusted_mutual_info_score(y, labels))

In [28]:
metr(labels, X_cv)
#самое приличное, что вышло(пробовал еще tfidf, и прибавлял разные классификаторы):

Silhouette Coefficient: 0.020
Homogeneity: 0.922
Completeness: 0.064
V-measure: 0.119
Adjusted Rand Index: 0.000
Adjusted Mutual Information: -0.001


## **Поиск аутлаеров в данных с помощью кластеризации (нужно найти хотя бы 20 странных не одинаковых текста) **

In [37]:
cluster = MeanShift(cluster_all=False, bandwidth=0.5)
cluster.fit(X_svd)
labels = cluster.labels_

In [38]:
miniX['cluster_num'] = labels

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [41]:
miniX[miniX.cluster_num==-1]

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,cluster,cluster_num
223,008faa76dd3eb890,i only deleted personal attacks,0,0,0,0,0,0,non_toxic,-1


In [82]:
cluster = MeanShift(cluster_all=False, bandwidth=0.7,min_bin_freq=0.7)
cluster.fit(X_svd)
labels = cluster.labels_

In [83]:
miniX['cluster_num'] = labels

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [84]:
len(miniX[miniX.cluster_num==-1])

31

In [85]:
#Тут нашлось много странных, они все вообще выглядят странными.
miniX[miniX.cluster_num==-1]

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,cluster,cluster_num
101,003fa0c68deca750,Check the following websites:\n\nhttp://www.iranchamber.com/personalities/farabi/farabi.php\nhttp://www.islam.org.br/%C2%A0al_farabi.htm\nhttp://www.superbeyin.com/sohbet/sohbet.htm,0,0,0,0,0,0,non_toxic,-1
238,0097dd5c29bf7a15,u r a tw@ fuck off u gay boy.U r smelly.Fuck ur mum poopie,1,0,1,0,1,1,toxic obscene insult identity_hate,-1
355,00e3110f4ff4144f,There I've made him bold. Thats better. Lemmey talk,0,0,0,0,0,0,non_toxic,-1
702,01e82a7c3b00c42a,"Valerie Poxleitner \n\nValeri Poxleitner, A.K.A. Lights. If",0,0,0,0,0,0,non_toxic,-1
771,020dbbdb2f69bd97,"""Organizations \n|class= Start \n|importance= Low \n|maindykdate= \n|needs-infobox= Yes \n|needs-image= Yes \n|attention= \n}}\n{{WikiProject """,0,0,0,0,0,0,non_toxic,-1
1131,030f86c7115f78db,thanks \n\nthanks but i know,0,0,0,0,0,0,non_toxic,-1
1341,03a749e7639fd6be,I AM GAY\nLove\n \nI AM GAY\nLove\n \nI AM GAY\nLove,1,0,0,0,0,0,toxic,-1
1418,03d16c4bd7361b6f,"Appreciation \n\nHi there ACROTERION, from Portugal,\n\nplease read",0,0,0,0,0,0,non_toxic,-1
1531,041e780ba6322faf,"Schedule\nPractice started May 1\nTime trials May 11, 12, 18, 19\nRace day May 30\n\nhttp://news.google.com/newspapers?id=-20hAAAAIBAJ&sjid;=ZIcFAAAAIBAJ&pg;=5399,2528876&dq;=indianapolis+500&hl;=en",0,0,0,0,0,0,non_toxic,-1
1648,04740afdec5c0429,"God, I just hate them so much. Lets hope they die.",1,0,0,0,0,0,toxic,-1


In [63]:
cluster = DBSCAN(min_samples=5.1, eps=35.7, leaf_size=70,  ) 
cluster.fit(X_svd)
labels = cluster.labels_
miniX['cluster_num'] = cluster.labels_

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [64]:
print(len(miniX[miniX.cluster_num==-1]))

18


In [65]:
#Ну вот здесь большинство странные, за редким исключением:
miniX[miniX.cluster_num==-1]

id  \
788   0218713b96c82905   
1043  02d9ff88fc2893c8   
1308  0392452585b760e3   
2249  06168775082a96d2   
2349  064b57734ce5aad0   
2366  065b10e83e654132   
2420  067c5e814e88a56b   
2567  06e08f9a9acb421a   
2799  078a7beb1a1be141   
2920  07e36fc910bd3eec   
3090  0856c9ea94df1446   
3352  090a0d1ebb5d71b8   
4174  0b2055f13934a41e   
4557  0c1c007ab2d6d30f   
4712  0c7c341727488579   
4822  0cc5e89357e6d1be   
4910  0cfef9f9854da18f   
4994  0d39bef98c00f870   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [51]:
#еще несколько я выписал из разных кластеризаций:

In [143]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, LSTM, GRU, Bidirectional, TimeDistributed, InputLayer, Embedding, Conv1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras import layers

## **Любая нейронная модель (минимум 5 слоев) с Dropout, Pooling и колбеками **

In [54]:
!pip install pandas scikit-learn gensim matplotlib

In [55]:
import tensorflow as tf

In [56]:
from collections import Counter
import matplotlib.pyplot as plt
import gensim
%matplotlib inline

In [57]:
from tensorflow.keras import backend as K
def f1(y_true, y_pred):
    def recall(y_true, y_pred):

        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):

        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [104]:
def preprocess(text):
    tokens = text.lower().split()
    tokens = [token.strip(punctuation) for token in tokens]
    return tokens

In [105]:
miniX = my_train['comment_text'][:10000]
miniy = my_train['toxic'][:10000]

In [106]:
vocab = Counter()
for text in miniX:
    vocab.update(preprocess(text))

In [107]:
filtered_vocab = set()

for word in vocab:
    if vocab[word] > 50:
        filtered_vocab.add(word)

In [109]:
word2id = {'UNK':1, 'PAD':0}

for word in filtered_vocab:
    word2id[word] = len(word2id)

In [110]:
id2word = {i:word for word, i in word2id.items()}

In [91]:
X = []

for text in miniX:
    tokens = preprocess(text)
    ids = [word2id.get(token, 1) for token in tokens]
    X.append(ids)

In [111]:
MAX_LEN = max(len(x) for x in X)

In [112]:
X = tf.keras.preprocessing.sequence.pad_sequences(X, maxlen=MAX_LEN)

In [113]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.05)

In [94]:
y = miniy

In [95]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.05)

In [131]:
model = tf.keras.Sequential()

model.add(tf.keras.layers.Embedding(input_dim=len(word2id),
                                      input_length=MAX_LEN, output_dim=100)) 

model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy',
              metrics=['accuracy'])

In [132]:

checkpoint = tf.keras.callbacks.ModelCheckpoint('model.weights',  
                                                monitor='val_f1',
                                                verbose=1, 
                                                save_weights_only=True,
                                                save_best_only=True,
                                                mode='max', 
                                                save_freq='epoch' 
                                               )


early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_f1', 
                                              min_delta=0.01,
                                              patience=3, 
                                              verbose=1, 
                                              mode='max',
                                              )

In [133]:
model.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid),
          batch_size=2000,
         epochs=10,
         callbacks=[checkpoint, early_stop])

Epoch 1/10
5/5 [==============================] - 19s 4s/step - loss: 0.5271 - accuracy: 0.8285 - val_loss: 0.3150 - val_accuracy: 0.9060
Epoch 2/10
5/5 [==============================] - 19s 4s/step - loss: 0.3340 - accuracy: 0.9031 - val_loss: 0.3261 - val_accuracy: 0.9060
Epoch 3/10
5/5 [==============================] - 19s 4s/step - loss: 0.3188 - accuracy: 0.9032 - val_loss: 0.3183 - val_accuracy: 0.9060
Epoch 4/10
5/5 [==============================] - 19s 4s/step - loss: 0.3087 - accuracy: 0.9033 - val_loss: 0.3127 - val_accuracy: 0.9060
Epoch 5/10
5/5 [==============================] - 19s 4s/step - loss: 0.2999 - accuracy: 0.9034 - val_loss: 0.3117 - val_accuracy: 0.9060
Epoch 6/10
5/5 [==============================] - 19s 4s/step - loss: 0.2932 - accuracy: 0.9035 - val_loss: 0.3078 - val_accuracy: 0.9060
Epoch 7/10
5/5 [==============================] - 19s 4s/step - loss: 0.2842 - accuracy: 0.9038 - val_loss: 0.2982 - val_accuracy: 0.9060
Epoch 8/10
5/5 [==================

In [134]:
preds = model.predict(X_valid).reshape(-1)

In [135]:
print(classification_report(y_valid, (preds > 0.5).astype(int)))

              precision    recall  f1-score   support

           0       0.91      1.00      0.95       453
           1       0.00      0.00      0.00        47

    accuracy                           0.91       500
   macro avg       0.45      0.50      0.48       500
weighted avg       0.82      0.91      0.86       500



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [67]:
#плохо, попробую с эмбеддингами

## **Использование эмбедингов в модели (предобученные или обученные с нуля word2vec, fastext, glove) **

In [136]:
texts = my_train['comment_text'][:10000].apply(preprocess).tolist()

In [137]:
ft2 = gensim.models.FastText(texts, window=3,  size=100, iter=4)

In [138]:
ft = gensim.models.FastText(texts, size=100, iter=2)

In [139]:
weights = np.zeros((len(word2id), 100))

for word, i in word2id.items():
    if word == 'PAD':
        continue
    if word == 'UNK':
        weights[i] = ft['какоераоваыпаопрвлоа']   
    try:
        weights[i] = ft[word]       
    except KeyError:
        weights[i] = ft['опрагпллирао']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


In [140]:

weights2 = np.zeros((len(word2id), 100))

for word, i in word2id.items():
    if word == 'PAD':
        continue
    
    if word == 'UNK':
        weights2[i] = ft2['какоераоваыпаопрвлоа']
    
    try:
        weights2[i] = ft2[word]
    
    
    except KeyError:
        weights[i] = ft2['опрагпллирао']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  app.launch_new_instance()


In [144]:
inputs = tf.keras.layers.Input(shape=(100,))


embeddings = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=100, trainable=False,
                                      weights=[weights])(inputs, )
mean = tf.keras.layers.Lambda(lambda x: tf.keras.backend.mean(x,  axis=1))(embeddings)
dense = tf.keras.layers.Dense(64, activation='relu')(mean)
dense = tf.keras.layers.Dense(64, activation='relu')(mean)
outputs = tf.keras.layers.Dense(1, activation='linear')(dense)

embeddings2 = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=100, trainable=False,
                                      weights=[weights2])(inputs, )
mean2 = tf.keras.layers.Lambda(lambda x: tf.keras.backend.mean(x,  axis=1))(embeddings)
dense2 = tf.keras.layers.Dense(64, activation='relu')(mean)
dense2 = tf.keras.layers.Dense(64, activation='relu')(mean)
outputs2 = tf.keras.layers.Dense(1, activation='linear')(dense)

added = layers.Concatenate()([outputs, outputs2 ])
out = layers.Dense(64)(added)


model = tf.keras.Model(inputs=inputs, outputs=outputs)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer,
              loss='mse',
              metrics=[tf.keras.metrics.RootMeanSquaredError()])

In [146]:
model.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid),
          batch_size=1024,
         epochs=15)

Epoch 1/15
10/10 [==============================] - 2s 172ms/step - loss: 0.0904 - root_mean_squared_error: 0.3006 - val_loss: 0.0870 - val_root_mean_squared_error: 0.2949
Epoch 2/15
10/10 [==============================] - 2s 173ms/step - loss: 0.0898 - root_mean_squared_error: 0.2996 - val_loss: 0.0865 - val_root_mean_squared_error: 0.2942
Epoch 3/15
10/10 [==============================] - 2s 173ms/step - loss: 0.0893 - root_mean_squared_error: 0.2988 - val_loss: 0.0862 - val_root_mean_squared_error: 0.2936
Epoch 4/15
10/10 [==============================] - 2s 174ms/step - loss: 0.0889 - root_mean_squared_error: 0.2981 - val_loss: 0.0859 - val_root_mean_squared_error: 0.2930
Epoch 5/15
10/10 [==============================] - 2s 177ms/step - loss: 0.0885 - root_mean_squared_error: 0.2975 - val_loss: 0.0856 - val_root_mean_squared_error: 0.2926
Epoch 6/15
10/10 [==============================] - 2s 173ms/step - loss: 0.0882 - root_mean_squared_error: 0.2970 - val_loss: 0.0854 - val_

In [147]:
preds = model.predict(X_valid).reshape(-1)

In [148]:
print(classification_report(y_valid, (preds > 0.5).astype(int)))

              precision    recall  f1-score   support

           0       0.91      1.00      0.95       453
           1       0.00      0.00      0.00        47

    accuracy                           0.91       500
   macro avg       0.45      0.50      0.48       500
weighted avg       0.82      0.91      0.86       500



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Тоже плохо. 
# Результат почти каждого пункта моей работы мне не нравится, но я уже рискую пропустить дедлайн.

# Огромное спасибо за курс!